<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wandb
!wandb login

     |████████████████████████████████| 2.0MB 5.7MB/s 
     |████████████████████████████████| 133kB 31.2MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 163kB 27.2MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=498c4566abce21d04ed9a097346ba76b61234af03a68e97e8af71d7e786668f7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=41490cb54757523c5fe92431889a0ec5978f0db2ac78f18c24264473d7e80551
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: App

In [110]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import sklearn

wandb.init(project="assignment-1", entity="swathi")
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

In [4]:
def load_images():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train_img = x_train[:54000]
  X_validation_img = x_train[54000:]
  X_test_img = x_test
  return X_train_img, X_validation_img, X_test_img

In [166]:
def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train = np.transpose((np.array(x_train[:54000],dtype=int).reshape([54000,784]))/255)
  Y_train = np.transpose((np.array(y_train[:54000],dtype=int).reshape([54000,1])))
  X_validation = np.transpose((np.array(x_train[54000:],dtype=int).reshape([6000,784]))/255)
  Y_validation = np.transpose(np.array(y_train[54000:],dtype=int).reshape([6000,1]))
  X_test = np.transpose((np.array(x_test,dtype=int).reshape([10000,784]))/255)
  Y_test = np.transpose(np.array(y_test,dtype=int).reshape([10000,1]))
  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [223]:
def log_images():
  images = []
  labels = []
  for i in range(len(X_train_img)):
    if len(labels)==len(classes):
      break
    j = random.randint(0,53999)
    if classes[Y_train[j,0]] not in labels:
      images.append(X_train_img[j])
      labels.append(classes[Y_train[j,0]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

Loading data

In [224]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = load_data()

In [225]:
log_images()

IndexError: ignored

Defining basic parameters

In [226]:
N= [0,64,32,16,10]  #Insert the number of neurons in each layer
N[0]=X_input.shape[0]
l = len(N) - 1

Weights initialising

In [159]:
def initialize_weights(N):
  np.random.seed(0)
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    W.append(np.random.randn(N[i],N[i-1])*(10**-3))
    b.append(np.zeros((N[i],1)))

  return W,b
  

In [227]:
def one_hot(y):
  one_hot_y=np.zeros((10,X_input.shape[1]))
  for i in range(X_input.shape[1]):
    one_hot_y[y[0,i],i] = 1
  
  return one_hot_y

In [229]:
print (one_hot(Y_train)[:,0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


Softmax

In [230]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [231]:
def sigmoid(a):

  return 1/(1+np.exp(-a))


Sigmoid derivative

In [102]:
def dSigmoid(a):

  return sigmoid(a)*(1-sigmoid(a))

Feed forward network

In [135]:
def feedforward(X_input, W, b, N, l):
  a=[]
  a.append(0)
  h=[]
  h.append(X_input)
  for i in range(1,l):
    # print(i)
    #print (W[i].shape)
    #print (h[i-1].shape)
    #print (b[i].shape)
    a_new=b[i]+np.matmul(W[i],h[i-1])
    h_new=sigmoid(a_new)
    a.append(a_new)
    h.append(h_new)

  a_new=b[l]+np.matmul(W[l],h[l-1])
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

Backpropogation

In [190]:
def backrop(y,y_hat,W,a,h):
  grad_W=[]
  grad_b=[]
  grad_a=[]
  grad_h=[]
  
  da = y_hat - one_hot(y)
  grad_a.append(da)

  for i in reversed(range(1,l+1)):
    dw = np.matmul(da,np.transpose(h[i-1]))
    db = np.sum(da, axis=1, keepdims=True)
    print (db.shape)
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dSigmoid(a[i-1])
    
    grad_W.append(dw)
    grad_b.append(db)
    grad_a.append(da_prev)
    grad_h.append(dh_prev)
    da=da_prev

  grad_W.append(0)
  grad_b.append(0)
  grad_W.reverse()
  grad_b.reverse()
  grad_a.reverse()
  grad_h.reverse()
  

  return grad_W,grad_b,grad_a,grad_h


Gradient Descent

In [188]:
def grad_descent(W,b,grad_W,grad_b,learn_rate=0.01):
  W_new=[]
  W_new.append(0)
  b_new=[]
  b_new.append(0)
  for i in range(1,l+1):
    W_new.append(W[i] - learn_rate*grad_W[i])
    b_new.append(b[i] - learn_rate*grad_b[i])
  return W_new,b_new


Training

In [185]:
def training(N,l,epochs,X_input,y):
  W , b = initialize_weights(N)
  for j in range(epochs):
    a , h , y_hat = feedforward(X_input, W, b, N, l)
    print (y_hat.shape)
    #calculate_loss(y_hat,y) 
    grad_W , grad_b = backrop(y,y_hat,W,a,h)[0:2]
    W , b = grad_descent(W,b,grad_W,grad_b)
  return W,b


In [202]:
def predict(W,b,X_test,N,l):
  y_hat = feedforward(X_test, W, b, N, l)[2]
  y_class = np.argmax(y_hat,axis=0).reshape([1,10000])

  return y_class

In [232]:
def calculate_loss(y_hat,Y):
  loss = 0
  for i in range(Y.shape[1]):
    loss -= np.log(y_hat[Y[i,0],i])
  print (loss)

In [194]:
import sklearn.metrics

In [222]:
Y_predict = predict(w_new,b_new,X_test,N,l)
print (Y_predict[0,1:20])
print (Y_test[0,1:20])
#sklearn.metrics.accuracy_score(Y_test, Y_predict)

[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
[2 1 1 6 1 4 6 5 7 4 5 7 3 4 1 2 4 8 0]


In [132]:
def gradient_descent(epochs, learn_rate):
  W, b = initialize_weights()
  W_new = W
  b_new = b
  for i in range(epochs):
    a, h, y_hat = feed_forward(W,b)
    calculate_loss(y_hat, Y)
    grad_W, grad_b, grad_h, grad_a = backrop(Y, y_hat, W, a, h)
    for j in range(1,L+1):
      W_new[j] = W[j] - grad_W[j]
      b_new[j] = b[j] - grad_b[j]
    W = W_new
    b = b_new
    print (b_new[1].shape)
  a, h, y_hat = feed_forward(W,b)
  calculate_loss(y_hat, Y)

In [169]:
W, b = initialize_weights(N)
print (b[1].shape)

(64, 1)


In [234]:
w_new,b_new= training(N,l,1,X_input,Y_train)


(10, 54000)
(10, 1)
(16, 1)
(32, 1)
(64, 1)


In [175]:
print (w_new[4].shape)

(10, 16)


In [233]:
Y_hat_new = feedforward(X_train, w_new, b_new, N, l)[2]
print (Y_hat_new[0,:])
print (Y_train[0][1:5])
print (Y_hat_new[:,0])

[0.09975108 0.09975108 0.09975108 ... 0.09975108 0.09975108 0.09975108]
[0 0 3 0]
[0.09975108 0.10019722 0.09984383 0.09983017 0.09990902 0.09979819
 0.10021903 0.10022251 0.10003183 0.1001971 ]


In [235]:
Y_hat_new = feedforward(X_train, w_new, b_new, N, l)[2]

In [238]:
print (Y_hat_new[0,:])
print (Y_train[0,0:5])
print (Y_hat_new[:,0])

[0.02665096 0.02665096 0.02665096 ... 0.02665096 0.02665096 0.02665096]
[9 0 0 3 0]
[0.02665096 0.08009331 0.08424915 0.07522676 0.01499535 0.16514847
 0.19535497 0.31915084 0.02521797 0.01391221]
